### Environment Config

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

### Read Dataset

Data was preprocessed by other script available in: https://github.com/HWatanuki/Projeto_D3TOP/blob/main/Codigos/Data_preprocessing_v0.3.ipynb

In [2]:
file_path = 'https://media.githubusercontent.com/media/HWatanuki/Projeto_D3TOP/main/Datasets/processed/'
file_name = 'dataset_airbnb-processed-spacy_2023-04-13_03-28-09-439.csv'

df = pd.read_csv(file_path + file_name, dtype=str)

In [3]:
print(f'No Rows: {df.shape[0]}')
print(f'No Columns: {df.shape[1]}', end='\n\n')
display(df.head(3))

No Rows: 561
No Columns: 94



,address,isAvailable,isHostedBySuperhost,location/lat,location/lng,numberOfGuests,photos/0/caption,photos/0/pictureUrl,photos/0/thumbnailUrl,pricing/rate/amount,pricing/rate/amountFormatted,pricing/rate/currency,pricing/rate/isMicrosAccuracy,pricing/rateType,primaryHost/badges/0,primaryHost/badges/1,primaryHost/badges/2,primaryHost/firstName,primaryHost/hasInclusionBadge,primaryHost/hasProfilePic,primaryHost/id,primaryHost/isSuperHost,primaryHost/languages/0,primaryHost/listingsCount,primaryHost/memberSince,primaryHost/responseRate,primaryHost/responseTime,primaryHost/smartName,primaryHost/totalListingsCount,reviews/0/author/firstName,reviews/0/author/id,reviews/0/createdAt,reviews/0/id,reviews/0/rating,reviews/1/author/firstName,reviews/1/author/id,reviews/1/createdAt,reviews/1/id,reviews/1/rating,reviews/2/author/firstName,reviews/2/author/id,reviews/2/createdAt,reviews/2/id,reviews/2/rating,reviews/3/author/firstName,reviews/3/author/id,reviews/3/createdAt,reviews/3/id,reviews/3/rating,reviews/4/author/firstName,reviews/4/author/id,reviews/4/createdAt,reviews/4/id,reviews/4/rating,reviews/5/author/firstName,reviews/5/author/id,reviews/5/createdAt,reviews/5/id,reviews/5/rating,reviews/6/author/firstName,reviews/6/author/id,reviews/6/createdAt,reviews/6/id,reviews/6/rating,reviews/7/author/firstName,reviews/7/author/id,reviews/7/createdAt,reviews/7/id,reviews/7/rating,reviews/8/author/firstName,reviews/8/author/id,reviews/8/createdAt,reviews/8/id,reviews/8/rating,reviews/9/author/firstName,reviews/9/author/id,reviews/9/createdAt,reviews/9/id,reviews/9/rating,roomType,stars,url,name,primaryHost/about,reviews/0/comments,reviews/1/comments,reviews/2/comments,reviews/3/comments,reviews/4/comments,reviews/5/comments,reviews/6/comments,reviews/7/comments,reviews/8/comments,reviews/9/comments
0,"Jersey City, New Jersey, United States",true,false,40.7233,-74.03946,2,NaN,https://a0.muscache.com/im/pictures/miso/Hosting-53775685/original/0812b9a1-2d2e-4725-b51f-881045a939b2.jpeg?aki_policy=large,https://a0.muscache.com/im/pictures/miso/Hosting-53775685/original/0812b9a1-2d2e-4725-b51f-881045a939b2.jpeg?aki_policy=small,99999,"$99,999",USD,false,nightly,82 Reviews,Identity verified,NaN,Joes,false,true,245395267,false,中文 (简体),8,Joined in February 2019,NaN,NaN,Joes,14,Miranda,74889676.0,2022-06-20T19:58:54Z,6.536263641559459e+17,4,Abhimanyu,4800160.0,2022-05-23T18:57:26Z,6.333017075212511e+17,5,Robert,162871850.0,2022-05-06T17:28:10Z,6.209355874005248e+17,5,Yu,303996502.0,2022-01-09T21:55:09Z,5.362712073993113e+17,5,Mitesh,253080011.0,2021-12-26T20:27:02Z,5.260799932553211e+17,4.0,Arusha,103758126.0,2021-12-22T21:29:10Z,5.2321216515417446e+17,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Entire rental unit,4.67,https://www.airbnb.com/rooms/53775685,entire cozy unit mins manhattan,hello welcome love see world able help people feel right home,great location nice building love parking include,stay perfect mention listing location min walk newport path right opposite huge grocery store joes super kind early checkin late checkout,excellent place stay central fo travel nyc work travel new jersey stay months apartment place advertise joes excellent host,best airbnb stay get great view outside window comfortable convenient living experience settle great,nice apartment great locality apartment list definitely rate star airbnb pet hair carpet couch month old bit concerned have allergies look clean place joes approachable easy communicate,customer take chance trust listing apartment exceed expectations super clean aesthetically pleasing minimalistic vibe definitely luxury apartment building convenient location right shoprite near waterfront path host responsive questions thanks great stay,NaN,NaN,NaN,NaN
1,"New York, United States",true,false,40.70641,-74.0092,2,NaN,https://a0.muscache.com/im/pictures/e2388507-1f5f-4000-aa1b-d3b2279e682a.jpg?aki_policy=large,https://a0.muscache.com/im/pictures/e2388507-1f5f-4000

### Data preparation

In [4]:
# Join all reviews comments for each announcement record
df_s01 = df.copy()

cols_comments = df_s01.columns[df_s01.columns.str.contains('comments')].values
print(cols_comments)

df_s01['comments_list'] = df_s01.apply(lambda x: [str(c) for c in x[cols_comments].values if not pd.isna(c)], axis=1)
df_s01['comments_string'] = df_s01.apply(lambda x:"  ".join([str(c) for c in x[cols_comments].values if not pd.isna(c)]), axis=1)

['reviews/0/comments' 'reviews/1/comments' 'reviews/2/comments'
 'reviews/3/comments' 'reviews/4/comments' 'reviews/5/comments'
 'reviews/6/comments' 'reviews/7/comments' 'reviews/8/comments'
 'reviews/9/comments']


In [5]:
df_s02 = df_s01[['address', 'numberOfGuests', 'name', 'pricing/rate/amount', 'roomType', 'stars',  'comments_list', 'comments_string']].copy()
df_s02.fillna('', inplace=True)

In [6]:
df_s02['comments_string'].values

array(['great location nice building love parking include  stay perfect mention listing location min walk newport path right opposite huge grocery store joes super kind early checkin late checkout  excellent place stay central fo travel nyc work    travel new jersey stay months apartment place advertise joes excellent host  best airbnb stay get great view outside window comfortable convenient living experience settle great  nice apartment great locality apartment list definitely rate star airbnb pet hair carpet couch month old bit concerned have allergies look clean place joes approachable easy communicate  customer take chance trust listing apartment exceed expectations super clean aesthetically pleasing minimalistic vibe definitely luxury apartment building convenient location right shoprite near waterfront path host responsive questions thanks great stay',
       'place exactly describe perfect month stay clean organize location convenient close supermarket pharmacy restaurants etc 

In [7]:
df_s02['comments_list'][0]

['great location nice building love parking include',
 'stay perfect mention listing location min walk newport path right opposite huge grocery store joes super kind early checkin late checkout',
 'excellent place stay central fo travel nyc work    travel new jersey stay months apartment place advertise joes excellent host',
 'best airbnb stay get great view outside window comfortable convenient living experience settle great',
 'nice apartment great locality apartment list definitely rate star airbnb pet hair carpet couch month old bit concerned have allergies look clean place joes approachable easy communicate',
 'customer take chance trust listing apartment exceed expectations super clean aesthetically pleasing minimalistic vibe definitely luxury apartment building convenient location right shoprite near waterfront path host responsive questions thanks great stay']

In [8]:
# Numeric variables
covs_num = ['numberOfGuests', 'pricing/rate/amount', 'stars']

# only convert numeric columns
df_s02 = df_s02.assign(**df_s02[covs_num].apply(pd.to_numeric, errors='coerce'))

## Recommendation system - Content Based Filtering Technique 

We chose this technique because it's not necessary having interation historic. Content-based filtering uses item features to recommend other items similar to what the user likes, based on their previous actions or explicit feedback.

### Feature engineering

#### TF-IDF for get relevant words

In [9]:
max_words = 10000
vectoriser = TfidfVectorizer(max_features=max_words)
tfidf_result = vectoriser.fit_transform(df_s02['comments_string'])
feature_names = vectoriser.get_feature_names_out()
print(f'Data Transformed.')
print('No. of feature_words: ', len(feature_names))

Data Transformed.
No. of feature_words:  6968


#### Create a new variable containing more relevant words of reviews as the summary

In [10]:
def get_summary(row):
  '''
  Function that iterates through the comment (all reviews together), calculating a score for each sentence by summing the TF-IDF values for each word appearing in the sentence. 
  Normalize the sentence scores by dividing by the number of tokens in the sentence (to avoid bias in favor of longer sentences). 
  Then sort the sentences by their scores, and return the highest-scoring sentences as the summary. 
  The number of sentences returned corresponds to roughly 20% of the overall length of the comment.
  '''
  import math

  comment_id = row.name  
  comment_text = row['comments_string']  
  sent_scores = []
  if not len(comment_text):
    return ''

  for sentence in comment_text.split('  '):    
    score = 0    
    sent_tokens = sentence.split()
    if not len(sent_tokens):
      continue
    for token in (t for t in sent_tokens if t in feature_names):
        score += tfidf_result[comment_id, np.squeeze(np.where(feature_names == token))]      
    sent_scores.append((score / len(sent_tokens), sentence))

  summary_length = int(math.ceil(len(sent_scores) / 5))
  sent_scores.sort(key=lambda sent: sent[0], reverse=True)

  summary_parts = []
  for summary_sentence in sent_scores[:summary_length]:
    summary_parts.append(summary_sentence[1])
  return ' '.join(summary_parts)



In [ ]:
%%time
df_s02['relevant_words'] = df_s02.apply(lambda x: get_summary(x) , axis=1)
df_s02.sample(2)

CPU times: total: 36.3 s
Wall time: 37.5 s


,address,numberOfGuests,name,pricing/rate/amount,roomType,stars,comments_list,comments_string,relevant_words
538,"Brooklyn, New York, United States",8.0,modern midcentury townhouse williamsburg,400.0,Entire townhouse,5.00,"[jaime place absolutely perfect month family adults stay nyc holidaying sydney australia moment arrive feel like home location convenient manhattan boroughs subway ferry fantastic bars restaurants minutes walk away comfortable clean cool honestly couldn fault jaime perfect host didn want leave thanks make trip lifetime memorable, great place great location blast jaime available need, jaime place perfect stay month work trip amazing fully stock jaime amazing communicative host, jaimes place fantastic brownstone need longer stay interior modern clean definitely spacious nyc standards bit walk minutes day nice quieter street nights highly recommend, jamie responsive attentive great stay love building amenities, jaime wonderful accommodate responsive place stunning perfect place month stay toddler, enjoy house easy family, stay jaimes house expect beautiful house beautiful furnishings kitchen require place sparkle toe seven adults party plenty space jaime respond messages immediately helpful greet arrive see williamsburg delightful house wonderful location easily usual write review comment improve honestly perfect sure nt pass opportunity stay guarantee wo nt disappoint jaime think forget wash coffee pot morning EMOJIsad thank stay certainly recommend hope, thank jaime amazing accommodations incredible time stay beautiful home past weekend jaime go sure need house immaculate leave husband champagne know th birthday thank jaime wonderful host look forward go future, immaculately clean pretty convenient location great little patio area communication point nt run single issue highly recommend]",jaime place absolutely perfect month family adults stay nyc holidaying sydney australia moment arrive feel like home location convenient manhattan boroughs subway ferry fantastic bars restaurants minutes walk away comfortable clean cool honestly couldn fault jaime perfect host didn want leave thanks make trip lifetime memorable great place great location blast jaime available need jaime place perfect stay month work trip amazing fully stock jaime amazing communicative host jaimes place fantastic brownstone need longer stay interior modern clean definitely spacious nyc standards bit walk minutes day nice quieter street nights highly recommend jamie responsive attentive great stay love building amenities jaime wonderful accommodate responsive place stunning perfect place month stay toddler enjoy house easy family stay jaimes house expect beautiful house beautiful furnishings kitchen require place sparkle toe seven adults party plenty space jaime respond messages immediately helpful greet arrive see williamsburg delightful house wonderful location easily usual write review comment improve honestly perfect sure nt pass opportunity stay guarantee wo nt disappoint jaime think forget wash coffee pot morning EMOJIsad thank stay certainly recommend hope thank jaime amazing accommodations incredible time stay beautiful home past weekend jaime go sure need house immaculate leave husband champagne know th birthday thank jaime wonderful host look forward go future immaculately clean pretty convenient location great little patio area communication point nt run single issue highly recommend,jaime place perfect stay month work trip amazing fully stock jaime amazing communicative host great place great location blast jaime available need
190,"New York, United States",5.0,massive bed bath harlem gem,673.0,Entire rental unit,4.88,"[run construction company annual project harlem second year rent sjoerds place guys tell airbnbs hotels ve stay past years favorite place say lot great communication friendly clear easy understand instructions bs come airbnbs time represent good platform thank sjoerd, greatly enjoy stay shu place go feel home fantast

The matter of this strategy is because of the weight of entities in text. That's why we tested another type of summary

##### Other form to summarise phrases

In [16]:
!pip install sumy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 67.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21696 sha256=6654f24405b171f03d294b18f3e00a97ba8c4b49db998207031cd04eec8df9d4
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=8481fe79ca725e75fd641cc7a810c7b34b0ef1dbf665a565729d55a1c7da88e7
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506

In [19]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
import nltk
nltk.download('punkt')

def generate_lsa_summary(list_txt, sentences_count=1):
    text = '. '.join(list_txt)
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, sentences_count)
    summary_text = ' '.join(str(sentence) for sentence in summary)
    return summary_text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [20]:
df_s02['summary'] = df_s02['comments_list'].apply(generate_lsa_summary)
print(df_s02['summary'].values[0:3])

['stay perfect mention listing location min walk newport path right opposite huge grocery store joes super kind early checkin late checkout.'
 'place exactly describe perfect month stay clean organize location convenient close supermarket pharmacy restaurants etc chris wonderful host extremely attentive easy communicate definitely book'
 '']


This summary makes more sense for the objective

In [22]:
file_path = 'sample_data/processed/'
# file_name = 'dataset_airbnb-processed_2023-04-04_01-45-49-997.csv'
file_name = 'dataset_airbnb-processed-summary_2023-04-13_03-28-09-439.csv'

df_s02.to_csv(file_path+file_name, index=False)

#### ML - BERT Model

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.preprocessing import StandardScaler

In [ ]:
# Loading pre-trained bert
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Covariates to be used in the model
covs = ['address', 'numberOfGuests', 'name', 'pricing/rate/amount', 'roomType', 'stars', 'summary']
covs_txt =  [c for c in covs if c not in covs_num]

# User inputs
user_input = ['Jersey', 6, 'near', 300, 'Entire', 4.5, 'comfortable']
input_dict = dict(zip(covs, user_input))

# test sample
df_sample = df_s02.sample(100)

In [ ]:
## Function to use BERT and calcule cosine similarity for each text variable
def fn_encode_covs_txt(cov, df, input_dict):
    print(cov)
    
    # Creating the corpus
    vocabulary =list(df[cov].values)
    vocabulary.append(input_dict[cov])

    # Coding the query and recommended items
    encoded_inputs = tokenizer(vocabulary, padding=True, truncation=True, return_tensors='pt')

    # Getting the text representations
    with torch.no_grad():
        outputs = model(**encoded_inputs)
        query_embeddings = outputs.last_hidden_state[0, 0, :]  # Query representation
        item_embeddings = outputs.last_hidden_state[1:, 0, :]  # Representations of recommended items

    # Calculating the cosine similarity
    cosine_similarity = torch.nn.functional.cosine_similarity(query_embeddings.unsqueeze(0), item_embeddings, dim=1)

    return(cosine_similarity)

In [ ]:
#chamar fn_encode for all covs_txt
cosine_similarity = list(map(lambda x: fn_encode_covs_txt(x, df_sample, input_dict), covs_txt))

# Normalizing the numeric covariates
scaler = StandardScaler()
covs_scaled = scaler.fit_transform(df_sample[covs_num])
normalized_covariates = torch.tensor(covs_scaled)

# Incorporating all cosines into the similarity
prd = cosine_similarity[0]
for c in cosine_similarity[1:]:
    prd = prd * c 

# Incorporating the covariates into the similarity
weighted_similarity = prd * normalized_covariates.prod(dim=1)

# Ranking recommended items based on weighted similarity
df_result = df_sample.copy()
df_result['similarity'] = weighted_similarity
df_result.sort_values('similarity', ascending=False)

address
name
roomType
summary


,address,numberOfGuests,name,pricing/rate/amount,roomType,stars,comments_list,comments_string,relevant_words,summary,similarity
0,"Jersey City, New Jersey, United States",2.0,entire cozy unit mins manhattan,99999.0,Entire rental unit,4.67,"[great location nice building love parking include, stay perfect mention listing location min walk newport path right opposite huge grocery store joes super kind early checkin late checkout, excellent place stay central fo travel nyc work travel new jersey stay months apartment place advertise joes excellent host, best airbnb stay get great view outside window comfortable convenient living experience settle great, nice apartment great locality apartment list definitely rate star airbnb pet hair carpet couch month old bit concerned have allergies look clean place joes approachable easy communicate, customer take chance trust listing apartment exceed expectations super clean aesthetically pleasing minimalistic vibe definitely luxury apartment building convenient location right shoprite near waterfront path host responsive questions thanks great stay]",great location nice building love parking include stay perfect mention listing location min walk newport path right opposite huge grocery store joes super kind early checkin late checkout excellent place stay central fo travel nyc work travel new jersey stay months apartment place advertise joes excellent host best airbnb stay get great view outside window comfortable convenient living experience settle great nice apartment great locality apartment list definitely rate star airbnb pet hair carpet couch month old bit concerned have allergies look clean place joes approachable easy communicate customer take chance trust listing apartment exceed expectations super clean aesthetically pleasing minimalistic vibe definitely luxury apartment building convenient location right shoprite near waterfront path host responsive questions thanks great stay,travel new jersey stay months apartment place advertise joes excellent host nice apartment great locality apartment list definitely rate star airbnb pet hair carpet couch month old bit concerned have allergies look clean place joes approachable easy communicate,stay perfect mention listing location min walk newport path right opposite huge grocery store joes super kind early checkin late checkout.,1.115838
186,"Brooklyn, New York, United States",10.0,story townhouse brooklyn sleep,750.0,Entire home,4.49,"[great stay deborahs lovely house feel spacious clean beds super comfy check easy deborah respond quickly hope stay soon thanks, home spacious location great lovely coffeeshops restaurants close issues cleanliness primary bathroom hair stains floor stain bedding, airbnb great location reduce offseason rate worth price house totally fine good large group miss basic amenities like toilet paper coffee molding coffee machine hair stains upstairs bathroom misremember check time communication host little aggressive try negotiate late check, great neighborhood spacious house overall good experience great value definitely pay, beautiful home amazing location definitely come soon enjoy time city, great place stay big family big living room convenient metro stop, home perfect place family gather christmas new year eve quieter manhattan far subway lots great restaurants walking distance great room couch comfy place hang kitchen outfit glasses dishes need coffee appliances like renovate older home place quirks lack perfection make comfortable aren afraid feet couch, deborahs place good place family coast coast spend christmas togetherthe accommodations exactly picture location greatas note heating impossible control downstairs drafty chilly try warm downstairs upstairs bedrooms like saunas wear layers clothing warm downstairsi think great place stay group family probably better warmer months, star EMOJIunfortunately rate host low follow issues booking property sleep adults rooms available maybe adults adults kids result 

In [ ]:
print(df_result.head())

                     address  numberOfGuests  \
379  New York, United States             2.0   
480  New York, United States             2.0   
283  New York, United States             4.0   
27   New York, United States            16.0   
476  New York, United States             5.0   

                                          name  pricing/rate/amount  \
379  luxury apartment spectacular skyline view                550.0   
480          lux apt walk best th ave shopping                818.0   
283             new years nyc gold star resort                695.0   
27               nyc market week showroom loft               1750.0   
476                             downtown oasis                800.0   

                      roomType  stars  \
379         Entire rental unit   4.72   
480         Entire rental unit    NaN   
283              Room in hotel    NaN   
27                 Entire loft   5.00   
476  Entire serviced apartment    NaN   

                                     